In [1]:
from datetime import date
import pandas as pd
import numpy as np
import holidays

In [27]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})
# dim_fecha.info()

In [ ]:
dim_fecha['day_number_of_week'] = ((dim_fecha['date'].dt.day_of_week + 1) % 7) + 1
dim_fecha['day_number_of_month']=dim_fecha['date'].dt.day
dim_fecha['day_number_of_year']=dim_fecha['date'].dt.day_of_year
dim_fecha['week_number_of_year'] = dim_fecha['date'].apply(
    lambda d: ((d - pd.Timestamp(d.year, 1, 1)).days + pd.Timestamp(d.year, 1, 1).weekday() + 1) // 7 + 1
)
dim_fecha['month_number_of_year']=dim_fecha['date'].dt.month
dim_fecha['calendar_quarter']=dim_fecha['date'].dt.quarter
dim_fecha['calendar_year']=(((dim_fecha['date'].dt.month-1)//6)+1)
dim_fecha['calendar_semester']=dim_fecha['date'].dt.year
dim_fecha['fiscal_year']=dim_fecha['date'].apply(lambda d: d.year if d.month>=7 else d.year-1)
dim_fecha['fiscal_month']=dim_fecha['date'].apply(lambda d:((d.month)-7)%12+1)
dim_fecha['fiscal_quarter']=((dim_fecha['fiscal_month']-1)//3) + 1
dim_fecha['fiscal_semester']=((dim_fecha['fiscal_month']-1)//6) + 1

# dim_fecha.drop('fiscal_mont',inplace=True,axis=1)
dim_fecha




#,week_number_of_year,calendar_year,calendar_semester,fiscal_year




AttributeError: 'DatetimeProperties' object has no attribute 'week'

In [15]:
df = pd.DataFrame({
    'date': pd.to_datetime(['2023-02-15', '2023-07-10', '2023-12-25'])
})

df['month']=(df['date'].dt.month-1)//6+1
df

,date,month
0,2023-02-15,1
1,2023-07-10,2
2,2023-12-25,2


In [18]:
5%4


1